In [ ]:
#default_exp cli

In [ ]:
#export

from fastcore.script import *
from fastcore.basics import partialler
import hydra
from omegaconf import DictConfig, OmegaConf, errors
#from corradin_ovp_utils.prefect_flows.step1 import get_config_task
from typing import List
from pathlib import Path
import logging
from rich.logging import RichHandler
from prefect.cli.run import run as prefect_run
from prefect.tasks.shell import ShellTask
from prefect.tasks.prefect import create_flow_run
import click
log = logging.getLogger(__name__)
import sys
import os


In [ ]:
test = prefect_run(["--project", "ovp_dev", "--name", "OVP_step1_hydra", "--execute"],standalone_mode = False)

Looking up flow metadata... Done
Creating run for flow 'OVP_step1_hydra'... Done
└── Name: feathered-quetzal
└── UUID: f8a3f141-8bee-4d34-962f-d96dc343b63c
└── Labels: ['agentless-run-bc46800c']
└── Parameters: {'config_path': '/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/corradin_ovp_utils/conf/example_prefect_step1_filled_conf.yaml'}
└── Context: {}
└── URL: https://cloud.prefect.io/corradin-lab-mit/flow-run/f8a3f141-8bee-4d34-962f-d96dc343b63c
Executing flow run...
└── 18:59:08 | INFO    | Creating subprocess to execute flow run...
Flow run succeeded!


In [ ]:
#export
def get_full_path_str(original_working_dir, path):
    #if the path is already an absolute path, we just have to check if path exist
    unchanged_path = Path(path)
    
    if unchanged_path.is_absolute():
        absolute_path_to_test = unchanged_path
    else:
        absolute_path_to_test = original_working_dir/unchanged_path
   
    try:
        assert absolute_path_to_test.exists() == True
    except:
        log.exception(f"Path does not exist: {absolute_path_to_test}")
        raise RuntimeError
    return absolute_path_to_test.as_posix()

def rmtree(f: Path):
    if f.is_file():
        f.unlink()
    else:
        for child in f.iterdir():
            rmtree(child)
        nfs_files = [file for file in f.iterdir() if file.stem.startswith(".nfs")]
        for file in nfs_files:
            file.unlink()
        f.rmdir()
        
OmegaConf.register_new_resolver("file", lambda file_path_with_extension: Path(file_path_with_extension).stem, replace=True)
OmegaConf.register_new_resolver("dataset_initial", lambda dataset_full_name: dataset_full_name.split("_")[0], replace=True)
OmegaConf.register_new_resolver("dir", get_full_path_str, replace=True)

In [ ]:
Path('/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/new_pipeline_results/outputs/MS/MS/test_MS_chr22/demo/2021-12-15/07-01-50/.nfs0000001702a1419400000211').stem.startswith(".nfs")

True

In [ ]:
#export
shell_task = ShellTask("pwd", stream_output=True)


In [ ]:
#export

def abort_and_cleanup(result_path):
    logging.info("[bold red] Aborting and cleaning up...[/]")
    ShellTask().run(command = "rm -rf {result_path}")
    logging.info("[bold red] Done [/]")
    
    
def run_ovp_pipeline(cfg) -> None:
    log.info("[bold red blink] Starting OVP pipeline [/]")
    #logging.info(f"All configurations\n {OmegaConf.to_yaml(cfg)}")
    cfg_output_section = cfg.run.output
    logging.info(f"Output folder path = {cfg_output_section.output_folder_dir_full_path}")
    logging.info(f"Result path = {cfg_output_section.result_path}")
    logging.info(f"Full result path = {cfg_output_section.dir}")
    shell_task.run()
    logging.info(f"Pairs file absolute path: {cfg.run.input.pairs_file_full_path}")
    logging.info(f"Dataset: {cfg.dataset.name}")
    logging.info(f"All configurations: {OmegaConf.to_yaml(OmegaConf.to_object(cfg))}")
    logging.info(f"Run name: {cfg.run.run_name}")
    logging.info(f"Output folder: {cfg.output_folder_dir}. If this is not what you want then change the 'output_folder_dir' argument")
    
    if cfg.interactive:
        try:
            logging.info("[bold red] Check if all the configurations are correct. Press Enter to continue, Ctrl + C to quit and clean up[/]")
            input()
        except KeyboardInterrupt:
            cfg_output_section = cfg.run.output
           
            try:
                sys.exit(0)
            except SystemExit:
                os._exit(0)
           
    
    OmegaConf.save(OmegaConf.to_object(cfg), ".hydra/config_filled.yaml")
    
    step1_command = ["--project", "ovp_dev",
                     "--name", "OVP_step1_hydra",
                     "--param", "config_path=.hydra/config_filled.yaml",
                     "--run-name", f"step1_{cfg.run.run_name}" ]
    
    if cfg.run.submit == False:
        step1_command.append("--execute")
        
    prefect_run(step1_command,standalone_mode = False)
    
@hydra.main(config_path="conf/hydra_conf", config_name="config")
def run_ovp_pipeline_static(cfg):
    try:
        run_ovp_pipeline(cfg)
    except errors.MissingMandatoryValue:
        result_path = Path(cfg_output_section.dir)
        logging.info(f"[bold red] Removing directory {result_path.as_posix()}[/]")
   
        
if __name__ == "__main__":
    run_ovp_pipeline_static()
    
# @call_parse
# def run_ovp_pipeline_dynamic(config_path:str,     # Path to config
#         # pairs_file: str,       # TSV pairs file, contain columns [GWAS_id, outside_id, GWAS_chrom, outside_chrom] in that order
#         # overrides: List,       # overrides to the config
#         ): 
#     "Run the pipeline from scratch"
#     # cfg = get_config_task.run(pairs_file = pairs_file, config_path = config_path, overrides = overrides)
#     # print(cfg)
#     # print(OmegaConf.to_object(cfg))
#     hydra_main_func_templated = hydra.main(config_path = config_path, config_name = "config")
#     run_pipeline = hydra_main_func_templated(run_ovp_pipeline)
#     run_pipeline()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides [overrides ...]]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

In [ ]:
run_ovp_pipeline_static()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides [overrides ...]]
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

/lab/corradin_biobank/FOR_AN/OVP/corradin_ovp_utils/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
